# Examples from "Chiral discrimination in helicity-preserving Fabry-Pérot cavities"
These examples show how to calculate the data corresponding to the corrected figures published in the erratum [Phys. Rev. A 108, 069902 (2023)](https://doi.org/10.1103/PhysRevA.108.069902) for the article ["Chiral discrimination in helicity-preserving Fabry-Pérot cavities", Phys. Rev. A 107, L021501 (2023)](https://doi.org/10.1103/PhysRevA.107.L021501).

For simplicity, we do not include the plotting code to reproduce the exact figures, which feature more complex layouts, annotations, subfigures with cuts of the data, etc.

## Import modules

In [ ]:
import chiral_transfermatrix as ct
import numpy as np
import matplotlib.pyplot as plt

## Dielectric permeability $\varepsilon$ 

In [ ]:
def eps_DL(omega, epsinf, omegap, omega0=0, gamma=0):
    """Drude-Lorentz model for the dielectric function of a material."""
    eps = epsinf + omegap**2 / (omega0**2 - omega**2 - 1j * gamma * omega)
    return eps

## Erratum Fig. 1 and Fig. 2 (corresponding to Fig. 2 and Fig. 4 of the article)

In [ ]:
# use numpy broadcasting rules to scan over 3D grid of parameters, results will have indices [i_kappa, i_omega, i_omegap]
omegap = np.linspace(0, 1, 401) # in eV
omega  = np.linspace(1.65, 2.2, 2400)[:,None] # in eV
kappa  = np.linspace(0, -1e-3, 2)[:,None,None]

lambda_vac = 1239.8419843320028 / omega # in nm, "magic" constant is hc in eV*nm

eps_Ag  = eps_DL(omega, epsinf=4.77574276, omegap=9.48300763, omega0=0,   gamma=0.17486845)
eps_mol = eps_DL(omega, epsinf=2.89,       omegap=omegap,     omega0=1.9, gamma=0.1)

air_infty = ct.MaterialLayer(d=np.inf, eps=1)
air_thin  = ct.MaterialLayer(d=0,      eps=1)
Ag_mirror = ct.MaterialLayer(d=16,     eps=eps_Ag)
molecules = ct.MaterialLayer(d=130,    eps=eps_mol, kappa=kappa)

omegaPR = 2
gammaPR = 0.05
mirror_1 = ct.helicity_preserving_mirror(omega,omegaPR=omegaPR,gammaPR=gammaPR,enantiomer=False)
mirror_2 = ct.helicity_preserving_mirror(omega,omegaPR=omegaPR,gammaPR=gammaPR,enantiomer=True)

mls_Ag = ct.MultiLayerScatt([air_infty, Ag_mirror,          molecules, Ag_mirror,          air_infty], lambda_vac, theta0=0.)
mls_HP = ct.MultiLayerScatt([air_infty, mirror_1, air_thin, molecules, air_thin, mirror_2, air_infty], lambda_vac, theta0=0.)

# calculate ΔDCTs corresponding to the difference in DCTs between kappa=-1e-3 and kappa=0
# assign them to variables within the result objects mls_Ag and mls_HP
mls_Ag.ΔDCTs = mls_Ag.DCTs[1] - mls_Ag.DCTs[0]
mls_HP.ΔDCTs = mls_HP.DCTs[1] - mls_HP.DCTs[0]

## Erratum Fig. 1

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(13,4),layout="constrained")

for (ax, data, vmax, clabel) in zip(axs,
                                    [(mls_Ag.Tsp[1]+mls_Ag.Tsm[1])/2, mls_HP.Tsm[1], mls_HP.Tsp[1]],
                                    [0.52, 0.26, 1],
                                    ["$T$", "$T_-$", "$T_+$"]):
    im = ax.pcolormesh(omegap, omega, data, vmin=0, vmax=vmax, cmap="inferno", shading="gouraud", rasterized=True)
    cb = plt.colorbar(im,ax=ax,fraction=0.10,pad=0.01)
    cb.ax.set_title(clabel)
    ax.set_xlabel(r"$\omega_p \sqrt{f}$ (eV/$\hbar$)")
axs[0].set_ylabel(r"$\omega$ (eV/$\hbar$)");

## Erratum Fig. 2

In [ ]:
vmax = abs(mls_HP.ΔDCTs).max()
plt.pcolormesh(omegap, omega, mls_HP.ΔDCTs, vmin=-vmax, vmax=vmax, cmap='seismic', shading='gouraud', rasterized=True)
cb = plt.colorbar()
cb.ax.set_title(r'$\Delta\mathrm{DCT}$')
plt.xlabel(r"$\omega_p \sqrt{f}$ (eV/$\hbar$)")
plt.ylabel(r"$\omega$ (eV/$\hbar$)")
plt.xlim(0,0.55)
plt.ylim(1.85,2.1);